# convert labeling from resnet to yolo

In [ ]:
# category classification labeling(to yolov8)
import json
import os
from tqdm import tqdm
import shutil

# labeling class
dicts = {'탑':0, '블라우스':1, '티셔츠':2, '니트웨어':3, '셔츠':4, 
         '브라탑':5, '후드티':6, '청바지':7, '팬츠':8, '스커트':9, 
         '레깅스':10, '조거팬츠':11, '코트':12, '재킷':13, '점퍼':14, 
         '패딩':15, '베스트':16, '가디건':17, '짚업':18, '드레스':19, 
         '점프수트':20}

# path 
pj_path = '/Users/young/Documents/DL_project/model_project'   # project folder path
img_path = pj_path + '/images'   # img path
fashion_path = pj_path +'/fa'   # json labeling path
convert_path = pj_path + '/yolo_category_labeling'    # txt labeling 저장 폴더 path
wrong_img_path = pj_path + '/wrong_datasets/train/wrong_img'    # 라벨링 잘못 저장된 이미지 이동할 폴더 path
wrong_json_path = pj_path + '/wrong_datasets/train/wrong_json' # 라벨링 잘못 저장된 json 파일 이동할 폴더 path

# 폴더에 .DS_Store가 저장되어있음 -> 맥북에서 생성한 폴더면 자동 생성되어있는 듯 함
# Get file name 
li = [f for f in os.listdir(fashion_path) if f != '.DS_Store'] 
img_name = [f for f in os.listdir(img_path) if f != '.DS_Store']

for label in tqdm(li):
    with open(os.path.join(fashion_path, label), "r") as f:
        fa = json.load(f)
    
    if label.split('.')[0] + '.jpg' not in img_name:
        continue
    
    filename = str(fa['이미지 정보']['이미지 식별자']) + '.jpg'
    ann = fa['데이터셋 정보']['데이터셋 상세설명']['렉트좌표']
    img_w = fa['이미지 정보']['이미지 너비']
    img_h = fa['이미지 정보']['이미지 높이']

    for key, value in ann.items():
        if len(value[0]) > 0:
            # class_label = str(dicts[key])
            try:
                class_label = str(fa['데이터셋 정보']['데이터셋 상세설명']['라벨링'][key][0]['카테고리'])
                bbox = []
            except:
                # 라벨링이 잘못된 파일인 경우 이미지, json 파일 이동
                print(filename)
                shutil.move(os.path.join(img_path, filename), os.path.join(wrong_img_path, filename))
                shutil.move(os.path.join(fashion_path, filename.split('.')[0] + '.json'), os.path.join(wrong_json_path, filename.split('.')[0] + '.json'))
                break
                
            for i in value[0].values():
                bbox.append(float(i))
                
            x, y, w, h = list(map(float, bbox))
            
            # yolo 라벨 좌표형식으로 변환 (yolo 라벨은 x_center, y_center, w, h를 0~1 사이값으로 정규화하여 사용)
            x = (x + w/2) / img_w
            y = (y + h/2) / img_h
            w = w / img_w
            h = h / img_h

            # 클래스, 좌표 txt 파일로 저장
            with open(os.path.join(convert_path, filename.split('.')[0] + '.txt'), 'a') as t:
                try:
                    t.write(str(dicts[class_label]) + ' ' + str(x) + ' ' + str(y) + ' ' + str(w) + ' ' + str(h) + '\n')
                except:
                    print(filename)
                    os.remove(os.path.join(convert_path, filename.split('.')[0] + '.txt'))
                    shutil.move(os.path.join(img_path, filename), os.path.join(wrong_img_path, filename))
                    shutil.move(os.path.join(fashion_path, filename.split('.')[0] + '.json'), os.path.join(wrong_json_path, filename.split('.')[0] + '.json'))
                    break